# Context data for the sea level data explorer

In [1]:
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
from datetime import datetime

#get data from worldbank
from pandas_datareader import wb

import glob
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import json
import matplotlib.pyplot as plt

InteractiveShell.ast_node_interactivity = "all"
pd.options.display.max_rows = 200
#matplotlib.rcParams['svg.fonttype'] = 'none'


# Sea Level Data, Stations

In [2]:
stations = pd.read_csv('context_data/sealevel_viz_psmsl_1985_2015.csv', usecols=[2, 4, 5, 6, 7, 9])

stations = stations.drop_duplicates()

stations.rename(columns={'ID': 'id'}, inplace=True)

stations['id'].nunique()
stations.head()

513

,id,location,latitude,longitude,country,continent
0,1,Brest,48.383,-4.495,FRA,europe
31,61,Marseille,43.279,5.354,FRA,europe
62,453,Le Havre,49.482,0.106,FRA,europe
93,467,Cherbourg,49.651,-1.636,FRA,europe
124,468,Dunkerque,51.048,2.367,FRA,europe


In [3]:
tide_data = pd.read_csv('data/dataviz/sealevel_viz_psmsl_1985_2015.csv')

In [4]:
tide_data

,id,1985,1986,1987,1988,1989,1990,1991,1992,1993,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,1,0.0,-1.0,4.0,9.0,15.0,16.0,15.0,8.0,5.0,...,40.0,34.0,36.0,41.0,58.0,59.0,63.0,65.0,77.0,70.0
1,3,0.0,0.0,-4.0,-8.0,-4.0,-10.0,-24.0,-29.0,-41.0,...,32.0,32.0,30.0,38.0,42.0,47.0,53.0,52.0,27.5,3.0
2,5,0.0,-5.0,-8.0,-7.0,4.0,9.0,3.0,4.0,1.0,...,40.0,34.0,33.0,36.0,48.0,50.0,67.0,71.0,81.0,77.0
3,7,0.0,-16.0,-24.0,-29.0,-13.0,16.0,23.0,28.0,9.0,...,50.0,68.0,79.0,67.0,62.0,NaN,NaN,NaN,NaN,NaN
4,8,0.0,-11.0,-1.0,0.0,25.0,34.0,29.0,28.0,18.0,...,17.0,31.0,42.0,42.0,49.0,59.0,53.0,50.0,45.0,45.0
5,9,0.0,-5.0,-3.0,5.0,12.0,23.0,12.0,9.0,-5.0,...,14.0,22.0,31.0,28.0,32.0,34.0,26.0,21.0,27.0,33.0
6,10,0.0,13.0,1.0,-40.0,-50.0,-46.0,-55.0,-43.0,-23.0,...,-2.0,-12.0,-16.0,-17.0,-20.0,-29.0,-18.0,-17.0,-4.0,12.0
7,11,0.0,-13.0,-5.0,-9.0,18.0,28.0,20.0,19.0,9.0,...,36.0,51.0,59.0,55.0,58.0,63.0,51.0,43.0,34.0,35.0
8,12,0.0,10.0,23.0,7.0,-6.0,-6.0,-3.0,-3.0,12.0,...,67.0,70.0,78.0,92.0,101.0,113.0,132.0,132.0,131.0,116.0
9,14,0.0,-20.0,-33.0,-57.0,-24.0,9.0,7.0,28.0,14.0,...,-75.0,-43.0,-19.0,-39.0,-66.0,-53.0,-68.0,-97.0,-108.0,-67.0


In [5]:
stations.describe()

,id,latitude,longitude
count,513.000000,513.000000,513.000000
mean,825.904483,33.845446,25.302012
std,546.822134,27.489068,97.354383
min,1.000000,-65.246000,-177.360000
25%,368.000000,29.135000,-68.517000
50%,808.000000,38.317000,16.548000
75%,1253.000000,52.167000,129.292000
max,2316.000000,79.550000,178.423000


# Worldbank Data, Emissions

In [6]:
#get the data from worldbank, Stand: 7. April 2017

emissions = pd.read_csv('context_data/c02_emissions_worlbank.csv', usecols=[0, 1, 57])

#emissions['c02_rank'] = emissions['2013'].rank(ascending=False)
emissions.rename(columns={'Country Code': 'country_code'}, inplace=True)
emissions.rename(columns={'Country Name': 'country_name'}, inplace=True)
emissions.rename(columns={'2013': 'c02_emissions'}, inplace=True)

#emissions.sort_values(by='c02_rank').head()

emissions.head()

,country_name,country_code,c02_emissions
0,Aruba,ABW,8.515395
1,Afghanistan,AFG,0.693183
2,Angola,AGO,1.384496
3,Albania,ALB,1.662185
4,Andorra,AND,6.473848


# Sea Level Population Data, National Population Data

In [7]:
sealevel_pop = pd.read_csv('context_data/lecz-urban-rural-population-land-area-estimates_country-90m.csv')

In [8]:
sealevel_pop['ElevationZone'] = sealevel_pop['ElevationZone'].str.strip()
sealevel_pop['ISO3v10'] = sealevel_pop['ISO3v10'].str.strip()
pop = sealevel_pop.set_index(['ElevationZone'])
pop = pop.loc[['Elevations Less Than or Equal To 10m']]

pop = pop.reset_index()

pop_low_elevation = pop.groupby(by=['ISO3v10'])['Population2010'].sum()
pop_low_elevation = pop_low_elevation.to_frame(name = 'coastal_population2010_sum')
pop_low_elevation = pop_low_elevation.reset_index()

pop_low_elevation.head()

,ISO3v10,coastal_population2010_sum
0,ABW,21327
1,AGO,651360
2,AIA,3848
3,ALB,354132
4,ANT,46295


In [9]:
pop_total = sealevel_pop.set_index(['ElevationZone'])
pop_total = pop_total.loc[['Total National Population']]

pop_total = pop_total.reset_index()

pop_total = pop_total.groupby(by=['ISO3v10'])['Population2010'].sum()
pop_total = pop_total.to_frame(name = 'total_population2010_sum')
pop_total = pop_total.reset_index()

pop_total.head()

,ISO3v10,total_population2010_sum
0,ABW,108141
1,AGO,19618432
2,AIA,15645
3,ALB,3215988
4,ANT,202847


# Sea Level PSMSL Trend Data

In [10]:
trend_1985_2015 = pd.read_csv('context_data/trends_1985_2015.csv', usecols=[4,7])
trend_1985_2015.rename(columns={'trend': 'trend_1985_2015'}, inplace=True)
trend_1985_2015.head()

,trend_1985_2015,_id
0,2.385,1
1,3.531,7
2,1.476,8
3,1.484,9
4,2.190,10


In [10]:
pd.qcut(trend_1985_2015['trend_1985_2015'], 5)

0        (2.158, 3.131]
1        (3.131, 4.206]
2        (0.555, 2.158]
3        (0.555, 2.158]
4        (2.158, 3.131]
5        (0.555, 2.158]
6        (4.206, 9.444]
7        (2.158, 3.131]
8      [-20.451, 0.555]
9        (2.158, 3.131]
10       (2.158, 3.131]
11       (0.555, 2.158]
12       (2.158, 3.131]
13       (2.158, 3.131]
14       (2.158, 3.131]
15     [-20.451, 0.555]
16       (4.206, 9.444]
17     [-20.451, 0.555]
18       (0.555, 2.158]
19     [-20.451, 0.555]
20       (0.555, 2.158]
21       (3.131, 4.206]
22     [-20.451, 0.555]
23     [-20.451, 0.555]
24     [-20.451, 0.555]
25     [-20.451, 0.555]
26       (2.158, 3.131]
27     [-20.451, 0.555]
28     [-20.451, 0.555]
29       (2.158, 3.131]
30       (0.555, 2.158]
31       (0.555, 2.158]
32     [-20.451, 0.555]
33     [-20.451, 0.555]
34     [-20.451, 0.555]
35       (2.158, 3.131]
36       (0.555, 2.158]
37       (4.206, 9.444]
38       (0.555, 2.158]
39       (0.555, 2.158]
40       (3.131, 4.206]
41       (2.158,

In [11]:
trend_longest = pd.read_csv('context_data/trends_longest.csv', usecols=[4, 6, 12])
trend_longest.rename(columns={'trend': 'trend_longest'}, inplace=True)
trend_longest.rename(columns={'startYear': 'start_year'}, inplace=True)
trend_longest.head()

,start_year,trend_longest,_id
0,1807,0.97,1
1,1811,0.80,2
2,1968,1.73,3
3,1843,2.12,7
4,1849,1.42,8


# Risk Level Data

In [131]:
riskindex = pd.read_csv('context_data/riskindex.csv', usecols=[0,1,2,3])
riskindex.rename(columns={'Rank': 'risk_rank'}, inplace=True)
riskindex.head()

,risk_rank,Country,risk_index,risk_description
0,1,Vanuatu,36.28,very high
1,2,Tonga,29.33,very high
2,3,Philippines,26.70,very high
3,4,Guatemala,19.88,very high
4,5,Bangladesh,19.17,very high


# Create Sea Level Context Data

In [12]:
#Add emissions
sealevel_context = stations.merge(emissions, left_on='country', right_on='country_code', how='left')

sealevel_context.describe()

,id,latitude,longitude,c02_emissions
count,513.000000,513.000000,513.000000,500.000000
mean,825.904483,33.845446,25.302012,10.213933
std,546.822134,27.489068,97.354383,4.530086
min,1.000000,-65.246000,-177.360000,0.214119
25%,368.000000,29.135000,-68.517000,6.779624
50%,808.000000,38.317000,16.548000,9.764393
75%,1253.000000,52.167000,129.292000,13.532295
max,2316.000000,79.550000,178.423000,16.389758


In [13]:
#Add sealevel population
sealevel_context = sealevel_context.merge(pop_low_elevation, left_on='country', right_on='ISO3v10', how='left')

sealevel_context['id'].nunique()

513

In [14]:
sealevel_context.head()

,id,location,latitude,longitude,country,continent,country_name,country_code,c02_emissions,ISO3v10,coastal_population2010_sum
0,1,Brest,48.383,-4.495,FRA,europe,France,FRA,5.050483,FRA,2983590.0
1,61,Marseille,43.279,5.354,FRA,europe,France,FRA,5.050483,FRA,2983590.0
2,453,Le Havre,49.482,0.106,FRA,europe,France,FRA,5.050483,FRA,2983590.0
3,467,Cherbourg,49.651,-1.636,FRA,europe,France,FRA,5.050483,FRA,2983590.0
4,468,Dunkerque,51.048,2.367,FRA,europe,France,FRA,5.050483,FRA,2983590.0


In [15]:
# clean up data (remove country code)

sealevel_context = sealevel_context.drop('ISO3v10', 1)

In [16]:
#Add national population
sealevel_context = sealevel_context.merge(pop_total, left_on='country', right_on='ISO3v10', how='left')

In [17]:
sealevel_context.head()

,id,location,latitude,longitude,country,continent,country_name,country_code,c02_emissions,coastal_population2010_sum,ISO3v10,total_population2010_sum
0,1,Brest,48.383,-4.495,FRA,europe,France,FRA,5.050483,2983590.0,FRA,63125894.0
1,61,Marseille,43.279,5.354,FRA,europe,France,FRA,5.050483,2983590.0,FRA,63125894.0
2,453,Le Havre,49.482,0.106,FRA,europe,France,FRA,5.050483,2983590.0,FRA,63125894.0
3,467,Cherbourg,49.651,-1.636,FRA,europe,France,FRA,5.050483,2983590.0,FRA,63125894.0
4,468,Dunkerque,51.048,2.367,FRA,europe,France,FRA,5.050483,2983590.0,FRA,63125894.0


In [18]:
# clean up data (remove country code)

sealevel_context = sealevel_context.drop('ISO3v10', 1)
sealevel_context = sealevel_context.drop('country_name', 1)
sealevel_context.head()

,id,location,latitude,longitude,country,continent,country_code,c02_emissions,coastal_population2010_sum,total_population2010_sum
0,1,Brest,48.383,-4.495,FRA,europe,FRA,5.050483,2983590.0,63125894.0
1,61,Marseille,43.279,5.354,FRA,europe,FRA,5.050483,2983590.0,63125894.0
2,453,Le Havre,49.482,0.106,FRA,europe,FRA,5.050483,2983590.0,63125894.0
3,467,Cherbourg,49.651,-1.636,FRA,europe,FRA,5.050483,2983590.0,63125894.0
4,468,Dunkerque,51.048,2.367,FRA,europe,FRA,5.050483,2983590.0,63125894.0


In [19]:
#Add trend data for 1985 to 2015
sealevel_context = sealevel_context.merge(trend_1985_2015, left_on='id', right_on='_id', how='left')

In [23]:
# clean up data (remove _id)
sealevel_context = sealevel_context.drop('_id', 1)

In [21]:
#Add longest trend
sealevel_context = sealevel_context.merge(trend_longest, left_on='id', right_on='_id', how='left')

In [22]:
sealevel_context.head()

,id,location,latitude,longitude,country,continent,country_code,c02_emissions,coastal_population2010_sum,total_population2010_sum,trend_1985_2015,start_year,trend_longest,_id
0,1,Brest,48.383,-4.495,FRA,europe,FRA,5.050483,2983590.0,63125894.0,2.385,1807.0,0.97,1.0
1,61,Marseille,43.279,5.354,FRA,europe,FRA,5.050483,2983590.0,63125894.0,3.530,1885.0,1.32,61.0
2,453,Le Havre,49.482,0.106,FRA,europe,FRA,5.050483,2983590.0,63125894.0,1.984,1959.0,2.66,453.0
3,467,Cherbourg,49.651,-1.636,FRA,europe,FRA,5.050483,2983590.0,63125894.0,1.908,1975.0,1.47,467.0
4,468,Dunkerque,51.048,2.367,FRA,europe,FRA,5.050483,2983590.0,63125894.0,NaN,1957.0,1.96,468.0


In [143]:
#Add riskindex
sealevel_context = sealevel_context.merge(riskindex, left_on='country_name', right_on='Country', how='left')

sealevel_context.head()

,id,location,latitude,longitude,country,continent,country_name,country_code,c02_emissions,c02_rank,coastal_population2010_sum,total_population2010_sum,trend_1985_2015,start_year,trend_longest,_id,risk_rank,Country,risk_index,risk_description
0,1,Brest,48.383,-4.495,FRA,europe,France,FRA,5.050483,92.0,2983590.0,63125894.0,2.385,1807.0,0.97,1.0,152.0,France,2.62,very low
1,61,Marseille,43.279,5.354,FRA,europe,France,FRA,5.050483,92.0,2983590.0,63125894.0,3.530,1885.0,1.32,61.0,152.0,France,2.62,very low
2,453,Le Havre,49.482,0.106,FRA,europe,France,FRA,5.050483,92.0,2983590.0,63125894.0,1.984,1959.0,2.66,453.0,152.0,France,2.62,very low
3,467,Cherbourg,49.651,-1.636,FRA,europe,France,FRA,5.050483,92.0,2983590.0,63125894.0,1.908,1975.0,1.47,467.0,152.0,France,2.62,very low
4,468,Dunkerque,51.048,2.367,FRA,europe,France,FRA,5.050483,92.0,2983590.0,63125894.0,NaN,1957.0,1.96,468.0,152.0,France,2.62,very low


In [24]:
sealevel_context.describe()

,id,latitude,longitude,c02_emissions,coastal_population2010_sum,total_population2010_sum,trend_1985_2015,start_year,trend_longest
count,513.000000,513.000000,513.000000,500.000000,5.110000e+02,5.110000e+02,424.000000,458.000000,458.000000
mean,825.904483,33.845446,25.302012,10.213933,1.395220e+07,1.288884e+08,2.044123,1951.307860,1.376572
std,546.822134,27.489068,97.354383,4.530086,2.030402e+07,2.084985e+08,3.324252,30.451367,2.855400
min,1.000000,-65.246000,-177.360000,0.214119,3.160000e+02,2.338000e+03,-20.451000,1807.000000,-17.620000
25%,368.000000,29.135000,-68.517000,6.779624,1.300770e+06,1.139003e+07,0.965000,1938.250000,0.492500
50%,808.000000,38.317000,16.548000,9.764393,3.970153e+06,6.241743e+07,2.626000,1960.000000,1.730000
75%,1253.000000,52.167000,129.292000,13.532295,2.443121e+07,1.264972e+08,3.886500,1972.000000,2.917500
max,2316.000000,79.550000,178.423000,16.389758,1.633811e+08,1.347565e+09,9.444000,1986.000000,8.890000


In [25]:
#sealevel_context = sealevel_context.drop('Country', 1)
sealevel_context = sealevel_context.drop('country_code', 1)

In [26]:
sealevel_context = sealevel_context.drop('country_name', 1)

ValueError: labels ['country_name'] not contained in axis

In [27]:
sealevel_context.head()

,id,location,latitude,longitude,country,continent,c02_emissions,coastal_population2010_sum,total_population2010_sum,trend_1985_2015,start_year,trend_longest
0,1,Brest,48.383,-4.495,FRA,europe,5.050483,2983590.0,63125894.0,2.385,1807.0,0.97
1,61,Marseille,43.279,5.354,FRA,europe,5.050483,2983590.0,63125894.0,3.530,1885.0,1.32
2,453,Le Havre,49.482,0.106,FRA,europe,5.050483,2983590.0,63125894.0,1.984,1959.0,2.66
3,467,Cherbourg,49.651,-1.636,FRA,europe,5.050483,2983590.0,63125894.0,1.908,1975.0,1.47
4,468,Dunkerque,51.048,2.367,FRA,europe,5.050483,2983590.0,63125894.0,NaN,1957.0,1.96


In [28]:
#Add tide data

sealevel_context = sealevel_context.merge(tide_data, left_on='id', right_on='id', how='left')

In [29]:
sealevel_context['c02_emissions'] = sealevel_context['c02_emissions'].round(decimals=2)
sealevel_context['trend_1985_2015'] = sealevel_context['trend_1985_2015'].round(decimals=2)

sealevel_context['trend_longest'] = sealevel_context['trend_1985_2015'].round(decimals=2)
sealevel_context['start_year'] = sealevel_context['start_year'].astype(object)
sealevel_context['total_population2010_sum'] = sealevel_context['total_population2010_sum'].astype(object)
sealevel_context['coastal_population2010_sum'] = sealevel_context['coastal_population2010_sum'].astype(object)
#sealevel_context['risk_rank'] = sealevel_context['risk_rank'].astype(object)
#sealevel_context['c02_rank'] = sealevel_context['c02_rank'].astype(object)

In [30]:
sealevel_context.head()

,id,location,latitude,longitude,country,continent,c02_emissions,coastal_population2010_sum,total_population2010_sum,trend_1985_2015,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,1,Brest,48.383,-4.495,FRA,europe,5.05,2.98359e+06,6.31259e+07,2.38,...,40.0,34.0,36.0,41.0,58.0,59.0,63.0,65.0,77.0,70.0
1,61,Marseille,43.279,5.354,FRA,europe,5.05,2.98359e+06,6.31259e+07,3.53,...,56.0,53.0,56.0,70.0,113.0,109.0,119.0,126.0,135.0,100.0
2,453,Le Havre,49.482,0.106,FRA,europe,5.05,2.98359e+06,6.31259e+07,1.98,...,30.0,26.0,26.0,27.0,33.0,33.0,33.0,34.0,45.0,49.0
3,467,Cherbourg,49.651,-1.636,FRA,europe,5.05,2.98359e+06,6.31259e+07,1.91,...,22.0,16.0,19.0,23.0,34.0,34.0,36.0,35.0,43.0,35.0
4,468,Dunkerque,51.048,2.367,FRA,europe,5.05,2.98359e+06,6.31259e+07,NaN,...,4.0,5.0,10.0,8.0,9.0,6.0,1.0,0.0,7.0,12.0


In [31]:
sealevel_context.describe()

,id,latitude,longitude,c02_emissions,trend_1985_2015,trend_longest,1985,1986,1987,1988,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
count,513.000000,513.000000,513.000000,500.000000,424.000000,424.000000,513.0,513.000000,513.000000,513.000000,...,513.000000,513.000000,513.000000,513.000000,513.000000,508.000000,499.000000,467.000000,444.000000,420.000000
mean,825.904483,33.845446,25.302012,10.213260,2.044340,2.044340,0.0,-2.649123,-2.419103,-6.970760,...,28.447693,32.070175,34.288499,34.545809,37.473684,41.968504,44.885772,47.173448,49.167793,50.947619
std,546.822134,27.489068,97.354383,4.530425,3.324125,3.324125,0.0,12.567850,19.534069,26.652882,...,85.357201,84.417992,81.105753,83.834206,90.029052,92.650107,98.271206,107.619950,110.761063,107.119152
min,1.000000,-65.246000,-177.360000,0.210000,-20.450000,-20.450000,0.0,-39.000000,-57.000000,-79.000000,...,-357.000000,-371.000000,-387.000000,-409.000000,-427.000000,-451.000000,-469.000000,-506.000000,-520.000000,-538.000000
25%,368.000000,29.135000,-68.517000,6.780000,0.967500,0.967500,0.0,-10.000000,-14.000000,-19.000000,...,-5.000000,-2.000000,2.000000,-3.000000,-3.000000,0.750000,3.500000,3.500000,7.750000,11.000000
50%,808.000000,38.317000,16.548000,9.760000,2.625000,2.625000,0.0,-4.000000,-2.000000,-4.000000,...,34.000000,37.000000,39.000000,42.000000,48.000000,53.500000,53.000000,64.000000,67.000000,67.000000
75%,1253.000000,52.167000,129.292000,13.530000,3.890000,3.890000,0.0,3.000000,7.000000,8.000000,...,68.000000,71.000000,73.000000,77.000000,85.000000,94.000000,102.000000,111.000000,112.000000,112.000000
max,2316.000000,79.550000,178.423000,16.390000,9.440000,9.440000,0.0,76.000000,114.000000,131.000000,...,543.000000,496.000000,448.000000,370.000000,364.000000,399.000000,411.000000,406.000000,412.000000,390.000000


In [32]:
sealevel_context.to_csv('context_data/sealevel_context_data.csv', index=False)

In [33]:
sealevel_context.to_csv('data/dataviz/sealevel_context_data.csv', index=False)